In [82]:
import os 
import pandas as pd

folder_path = os.getcwd()
print(folder_path)
print(os.listdir(folder_path))

C:\Users\aorjue01\Restatement\Product
['.ipynb_checkpoints', 'DollarSales_variancexlsx', 'Pending Standard Hierarchy After Product Restatement.xlsx', 'Pending Standard Hierarchy Before Product Restatement.xlsx', 'restatment_check.ipynb']


In [83]:
def data_check(file_name_b,file_name_a,sn,ts,c_name):
    fp_after = os.path.join(folder_path , file_name_a)
    fp_before =  os.path.join(folder_path , file_name_b)
    
    raw_before = pd.read_excel(fp_before,sheet_name = sn ,skiprows=ts)
    raw_after = pd.read_excel(fp_after,sheet_name = sn ,skiprows=ts)
    rd_with_na = raw_before.shape[0] - raw_after.shape[0]

    print(f"Total row in before table: {raw_before.shape[0]}")
    print(f"Total row in after table: {raw_after.shape[0]}")
    print(f"Row difference in tables: {rd_with_na}")
    print("==="*10)
    print(raw_before.columns)
    print(raw_after.columns)
    print("==="*10)

    raw_before["Product"] = raw_before["Product"].str.strip()
    raw_after["Product"] = raw_after["Product"].str.strip()
    
    mask_before = ~raw_before[c_name].isna()
    mask_after = ~raw_after[c_name].isna()
    
    before_no_na = raw_before[mask_before]
    after_no_na = raw_after[mask_after]
    before_no_na = before_no_na.set_index("Product") 
    after_no_na = after_no_na.set_index("Product")
    rd_without_na = before_no_na.shape[0] - after_no_na.shape[0]
    rd_without_na
    
    print(f"Total row in before table droping N/A: {before_no_na.shape[0]}")
    print(f"Total row in after table droping N/A: {after_no_na.shape[0]}")
    print(f"Row difference after droping N/A: {rd_without_na}")
    print("==="*10)
    compare_df = before_no_na.join(after_no_na, on= "Product",lsuffix ="_before")

    c_name_b = c_name + "_before"
    
    compare_df["difference"] = compare_df[c_name_b] - compare_df[c_name]
    compare_df["variance"] = (compare_df["difference"] / compare_df[c_name_b]) * 100
    
    comparation = compare_df[[c_name_b,c_name,"difference","variance"]]
    comparation_outrange_3p = comparation[ (comparation["variance"] > 3) | (comparation["variance"] < -3)]
    comparation_outrange_5p = comparation[ (comparation["variance"] > 5) | (comparation["variance"] < -5)]

    return raw_before , raw_after, comparation , comparation_outrange_3p,comparation_outrange_5p

In [84]:
def missing_rows(r_before, r_after):
    products_removed =  r_before[~r_before["Product"].isin(r_after["Product"])]
    products_included = r_after[~r_after["Product"].isin(r_before["Product"])]

    return products_removed["Product"], products_included["Product"]

In [85]:
def save_excel(name_file, sheet_name,dataframe,include_idx):
    name = name_file + ".xlsx"
    sn = sheet_name
    with pd.ExcelWriter(name, engine="openpyxl") as writer:
        dataframe.to_excel(writer, sheet_name=sn, index=include_idx)


In [86]:
file_name_a = "Pending Standard Hierarchy After Product Restatement.xlsx"
file_name_b = "Pending Standard Hierarchy Before Product Restatement.xlsx"
sheet_name = "1"
table_start = 7
column_to_check = 'Dollar Sales'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 16363
Total row in after table: 37094
Row difference in tables: -20731
Index(['Product', 'Dollar Sales', 'Unit Sales', 'Volume Sales',
       'Custom Category Value', 'Custom Sub Category Value',
       'Custom Flavor Value', 'Custom Size Range Value'],
      dtype='object')
Index(['Product', 'Dollar Sales', 'Unit Sales', 'Volume Sales',
       'Custom Category Value', 'Custom Sub Category Value',
       'Custom Flavor Value', 'Custom Size Range Value'],
      dtype='object')
Total row in before table droping N/A: 9266
Total row in after table droping N/A: 20672
Row difference after droping N/A: -11406


In [96]:
comp.head(29)

,Dollar Sales_before,Dollar Sales,difference,variance
Product,,,,
Standard Hierarchy_PENDING,6.207790e+10,NaN,NaN,NaN
CARBONATED BEVERAGES,1.748663e+10,1.748583e+10,7.923232e+05,4.531024e-03
COFFEE,6.508592e+09,6.525399e+09,-1.680735e+07,-2.582333e-01
COFFEE CREAMER - SS,3.642666e+08,3.634027e+08,8.639821e+05,2.371840e-01
COFFEE CREAMER - SS,3.642666e+08,3.634027e+08,8.639821e+05,2.371840e-01
COFFEE FILTERS,1.021726e+08,1.019459e+08,2.267194e+05,2.218985e-01
COFFEE FILTERS,1.021726e+08,1.019459e+08,2.267194e+05,2.218985e-01
TEA - BAGS/LOOSE,6.414760e+08,6.399218e+08,1.554235e+06,2.422905e-01
TEA - BAGS/LOOSE,6.414760e+08,6.255817e+08,1.589430e+07,2.477771e+00


In [88]:
comp_or_3p

,Dollar Sales_before,Dollar Sales,difference,variance
Product,,,,
DRINK MIXES,1.393496e+09,5.966300e+08,7.968664e+08,5.718468e+01
ASEPTIC JUICES,1.061396e+09,3.994501e+08,6.619460e+08,6.236560e+01
SPORTS DRINKS,3.525959e+09,4.228851e+09,-7.028921e+08,-1.993478e+01
BABY FOOD,2.612000e+01,2.830226e+06,-2.830200e+06,-1.083537e+07
PREMIXED COCKTAILS/COOLERS/HARD SELTZERS,1.366263e+06,1.240119e+06,1.261444e+05,9.232803e+00
...,...,...,...,...
WOMENS BEST WEIGHT CONTROL/NUTRITIONALS LIQ/PWD,1.294100e+02,1.164100e+02,1.300000e+01,1.004559e+01
WOMENS BEST WEIGHT CONTROL/NUTRITIONALS LIQ/PWD,1.294100e+02,1.164100e+02,1.300000e+01,1.004559e+01
ZHOU WEIGHT CONTROL/NUTRITIONALS LIQ/PWD,6.815125e+04,4.645968e+04,2.169158e+04,3.182858e+01


In [89]:
comp_or_5p = comp_or_5p.sort_values(by = "variance", ascending = False)

In [90]:
comp_or_5p.head(50)

,Dollar Sales_before,Dollar Sales,difference,variance
Product,,,,
MTN DEW CANNED JUICE DRINKS,1.333475e+07,1.350009,1.333475e+07,99.999990
DOLE CANNED/BOTTLED PINEAPPLE,9.660503e+07,45.000129,9.660498e+07,99.999953
DOLE CANNED/BOTTLED PINEAPPLE,9.660503e+07,45.000129,9.660498e+07,99.999953
SUNSWEET CANNED FRUIT JUICE,7.552015e+06,5.281538,7.552009e+06,99.999930
SUNSWEET SS BOTTLED PRUNE/FIG JUICE,3.593978e+07,30.156708,3.593975e+07,99.999916
BAI FLAVORED CONVENIENCE/BOTTLED STILL WATER,9.996263e+07,95.840000,9.996253e+07,99.999904
FANTA LOW CALORIE SOFT DRINKS,1.691245e+07,56.252017,1.691239e+07,99.999667
NOBRAND COFFEE FILTERS,1.861175e+06,12.281999,1.861162e+06,99.999340
V8 SS BOTTLED FRUIT DRINKS,5.802237e+07,891.467225,5.802148e+07,99.998464


In [91]:
comp_or_5p.tail(50)

,Dollar Sales_before,Dollar Sales,difference,variance
Product,,,,
SPARKLETTS NONFLAVORED JUG/BULK STILL WATER,7957.585010,3.173973e+05,-3.094397e+05,-3.888614e+03
TROPICAL REGULAR SOFT DRINKS,1679.342439,6.713945e+04,-6.546011e+04,-3.897961e+03
TROPICAL REGULAR SOFT DRINKS,1679.342439,6.713945e+04,-6.546011e+04,-3.897961e+03
DURTY GURL LIQUID COCKTAIL MIXES,4374.288998,1.817348e+05,-1.773605e+05,-4.054614e+03
DURTY GURL LIQUID COCKTAIL MIXES,4374.288998,1.817348e+05,-1.773605e+05,-4.054614e+03
DE MI PAIS CANNED JUICE DRINKS,4782.203782,2.210614e+05,-2.162792e+05,-4.522584e+03
DE MI PAIS CANNED JUICE DRINKS,4782.203782,2.210614e+05,-2.162792e+05,-4.522584e+03
KEURIG SINGLE CUP COFFEE,90501.654557,5.137142e+06,-5.046641e+06,-5.576297e+03
CALPICO FLAVORED CONVENIENCE/BOTTLED STILL WATER,3995.352441,2.323589e+05,-2.283636e+05,-5.715730e+03


In [92]:
prod_remov, prod_incl = missing_rows(raw_before,raw_after) 

In [93]:
prod_remov

0                               Standard Hierarchy_PENDING
17                         RTD SS MILK & MILK ALTERNATIVES
68                                   SS ENERGY DRINK MIXES
69                                   SS SPORTS DRINK MIXES
125                  FZ CRANBERRY JCE/DK/BLEND CONCENTRATE
                               ...                        
14023           VEGA ALL OTHER SNACK/GRANOLA BARS/CLUSTERS
14031      SUPER DIRT BREAKFAST/CEREAL/SNACK BARS/CLUSTERS
14077    NOT AVAILABLE NUTRITIONAL/INTRINSIC HEALTH VAL...
15112                 NOT AVAILABLE CANNED/BOTTLED PEACHES
15414                    MAISON PERRIER PREMIXED COCKTAILS
Name: Product, Length: 308, dtype: object

In [94]:
prod_incl

15                                   NON FRUIT DRINKS - SS
24                                         CHOCOLATE CANDY
32                                     JUICES/DRINKS - RFG
34                                                     NaN
111                                    SS ENERGY DRINK MIX
                               ...                        
37070    VEGA SPORT ENERGIZER WEIGHT CONTROL/NUTRITIONA...
37081    ZEAL NATURALS WEIGHT CONTROL/NUTRITIONALS LIQ/PWD
37084     ZHOU LITE UP WEIGHT CONTROL/NUTRITIONALS LIQ/PWD
37085    ZHOU LITE UP XTRA WEIGHT CONTROL/NUTRITIONALS ...
37089    ZANTREX SKINNYSTIX WEIGHT CONTROL/NUTRITIONALS...
Name: Product, Length: 6400, dtype: object

In [95]:
save_excel("DollarSales_variance","restatement_check",comp,True)

In [98]:
save_excel("Products removed","removed",prod_remov,False)
save_excel("Products included","included",prod_incl,False)

In [101]:
column_to_check = 'Unit Sales'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 16363
Total row in after table: 37094
Row difference in tables: -20731
Index(['Product', 'Dollar Sales', 'Unit Sales', 'Volume Sales',
       'Custom Category Value', 'Custom Sub Category Value',
       'Custom Flavor Value', 'Custom Size Range Value'],
      dtype='object')
Index(['Product', 'Dollar Sales', 'Unit Sales', 'Volume Sales',
       'Custom Category Value', 'Custom Sub Category Value',
       'Custom Flavor Value', 'Custom Size Range Value'],
      dtype='object')
Total row in before table droping N/A: 9266
Total row in after table droping N/A: 20672
Row difference after droping N/A: -11406


In [103]:
comp_or_5p.head(29)

,Unit Sales_before,Unit Sales,difference,variance
Product,,,,
DRINK MIXES,4.433889e+08,3.353394e+08,1.080496e+08,2.436902e+01
ASEPTIC JUICES,2.592743e+08,7.907494e+07,1.801993e+08,6.950143e+01
SPORTS DRINKS,1.012933e+09,1.097008e+09,-8.407532e+07,-8.300187e+00
BABY FOOD,2.000000e+00,1.539441e+06,-1.539439e+06,-7.697193e+07
PREMIXED COCKTAILS/COOLERS/HARD SELTZERS,1.592943e+05,1.390027e+05,2.029159e+04,1.273842e+01
YOGURT,4.707047e+05,5.558705e+05,-8.516584e+04,-1.809326e+01
TEA/COFFEE - RFG,6.278084e+05,6.702755e+05,-4.246717e+04,-6.764352e+00
N/A,5.491007e+05,1.804679e+05,3.686328e+05,6.713392e+01
CARROTS,4.601637e+04,4.329920e+04,2.717176e+03,5.904802e+00


In [104]:
save_excel("UnitSales_variance","restatement_check",comp,True)
save_excel("UnitSales_variance_5percent","restatement_check",comp_or_5p,True)